In [ ]:
!pip install googlemaps

!pip install ortools

!pip install gmaps

## Functions and imports

In [ ]:
import pandas as pd
import numpy as np
import folium
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import haversine_distances
from math import radians
import matplotlib.cm as cm
import warnings
from folium import plugins
import matplotlib.colors as colors

import googlemaps
from itertools import tee,islice
import time
import ast
import json
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import matplotlib.pyplot as plt
import matplotlib
import collections
collections.Iterable = collections.abc.Iterable
import warnings
warnings.filterwarnings("ignore")
from google.colab import output
output.enable_custom_widget_manager()
import gmaps
import gmaps.datasets

def combine_lat_lon(df):
    """This function combines latitude and longitude columns from a dataframe
     into a single column named 'coordinates'."""
    df['coordinates'] = list(zip(df['latitude'], df['longitude']))
    return df

def run_kmeans(orders_df, vehicle_df,num_clusters):
    """This function performs KMeans clustering on the coordinates of orders.
     It assigns each order to a cluster and returns the updated orders dataframe along with the KMeans model."""
    kmeans = KMeans(n_clusters=num_clusters, n_init=30)
    kmeans.fit(list(orders_df['coordinates']))
    orders_df['cluster'] = kmeans.labels_
    return orders_df, kmeans

def print_clusters(orders_df, kmeans):
    """This function prints the orders in each cluster identified by KMeans clustering."""
    cluster_ids = np.unique(kmeans.labels_)
    for cluster_id in range(len(cluster_ids)):
        cluster_orders = orders_df[orders_df['cluster'] == cluster_id]['order_id']
        print(f"Cluster {cluster_id}: Orders {', '.join(cluster_orders.astype(str))}")

def plot_clusters(orders_df,depot_coordinates, filename='cluster_map.html'):
    """This function plots the clusters on a map using Folium library
    and saves it to an HTML file. It also marks a depot location on the map."""
    map_center = [np.mean(orders_df['latitude']), np.mean(orders_df['longitude'])]
    m = folium.Map(location=map_center, zoom_start=10, width=1200, height=800)
    colors = ['red', 'green', 'blue', 'yellow', 'cyan', 'magenta', 'black', 'orange']
    clusters = orders_df['cluster'].unique()
    cmap = cm.get_cmap('magma', len(clusters))

    for idx, row in orders_df.iterrows():
        color = cmap(clusters.tolist().index(row['cluster']))
        color_hex = "#{:02x}{:02x}{:02x}".format(int(color[0] * 255), int(color[1] * 255), int(color[2] * 255))

        folium.CircleMarker(location=row['coordinates'], color=color_hex, fill=True, fill_opacity=1, radius=10, tooltip=f"Order ID: {row['order_id']}").add_to(m)
        # rm if dont want order id
        folium.Marker(location=row['coordinates'], popup=f"Order ID: {row['order_id']}", icon=None).add_to(m)

        folium.Marker(location=row['coordinates'], icon=folium.DivIcon(html=f"<div style='font-size: 12pt; color: black; position: fixed; right: +10px;'>{row['cluster']}</div>")).add_to(m)

    folium.Marker(location=depot_coordinates, icon=folium.Icon(color='red'), tooltip="Depot").add_to(m)
    m.save(filename)
    return m

def haversine(point1, point2):
    """Calculate haversine distance between two points given their latitude and longitude coordinates."""
    point1 = [radians(coord) for coord in point1]
    point2 = [radians(coord) for coord in point2]
    distance = haversine_distances([point1, point2])
    return distance[0][1] * 6371000/ 1609.34 # multiply by Earth radius to get meters

def select_nearest_cluster(orders_df, leaving_point):
    """This function selects the nearest cluster to a given leaving point (typically the current location of a vehicle)."""
    orders_df['distance_to_leaving_point'] = orders_df['coordinates'].apply(lambda x: haversine(x, leaving_point))
    nearest_cluster = orders_df.loc[orders_df['distance_to_leaving_point'].idxmin()]
    id = nearest_cluster['cluster']
    print(f'\n\n going to nearest cluster {id}')
    return nearest_cluster

def get_nearest_cluster_orders(orders_df,current_point):
    """ This function retrieves orders from the nearest cluster to a given current point."""
    nearest_cluster = select_nearest_cluster(orders_df, current_point)
    print(f"trying to get orders from nearest cluster \n {nearest_cluster['cluster']}")
    nearest_cluster_orders = orders_df[orders_df['cluster'] == nearest_cluster['cluster']]
    nearest_cluster_orders = nearest_cluster_orders.sort_values(by='distance_to_leaving_point')
    return nearest_cluster, nearest_cluster_orders

def assign_vehicle(nearest_cluster, vehicle_df):
    """This function assigns the first vehicle from the vehicle dataframe to the nearest cluster."""
    vehicle_id = vehicle_df.iloc[0]['vehicle_id']
    vehicle_capacity = vehicle_df.iloc[0]['vehicle_capacity ']
    return vehicle_id, vehicle_capacity

def nearest_cluster_collection(nearest_cluster_orders,exceeds_weight_orders,remaining_capacity,collected_orders,current_point):
    """ This function collects orders from the nearest cluster to a given current point, considering vehicle capacity constraints."""
    while len(nearest_cluster_orders) >=1:
      # current_point = (order['latitude'],order['longitude'])
      nearest_cluster_orders['distance_to_leaving_point'] = nearest_cluster_orders['coordinates'].apply(lambda x: haversine(x, current_point))
      nearest_cluster_orders = nearest_cluster_orders.sort_values(by='distance_to_leaving_point')
      order = nearest_cluster_orders.iloc[0]
      #display(order)
      nearest_cluster_orders= nearest_cluster_orders.drop(nearest_cluster_orders.index[0])

      if order['order_weight'] <= remaining_capacity:
          # print(f'net_weight: {net_weight}')
          current_order = order['order_id']
          current_weight = order['order_weight']
          collected_orders.append(current_order)
          remaining_capacity -= current_weight

          print(f'collected order {current_order} with weight {current_weight} .remaining capacity: {remaining_capacity}')
          current_point = (order['latitude'],order['longitude'])
      # if the order is > than the capacity
      # try to get the following orders of the cluster
      else:
          exceeds_weight_order = order['order_weight']
          exceeds_weight_order_id = order['order_id']
          # print(f'cant take {exceeds_weight_order} from order {exceeds_weight_order_id}')
          exceeds_weight_orders.append(exceeds_weight_order_id)

    return collected_orders,exceeds_weight_orders,remaining_capacity,current_point

def print_uncollected_orders(exceeds_weight_orders):
    """This function prints the uncollected orders (orders that couldn't be collected due to weight constraints)."""
    if len(exceeds_weight_orders) >= 1:
      print(f'could not collects orders: {exceeds_weight_orders}')
      exceeds_weight_orders = []
    else:
      print('collected all orders of cluster')
    return exceeds_weight_orders

def remove_collected_cluster(nearest_cluster,unique_clusters,valid_orders_df):
    """ This function removes the collected cluster from the list of unique clusters and updates the valid orders dataframe."""
    ncid = nearest_cluster['cluster']
    print(f'removing cluster {ncid} from {unique_clusters}')
    unique_clusters = np.delete(unique_clusters, unique_clusters==nearest_cluster['cluster'])
    valid_orders_df = valid_orders_df[valid_orders_df['cluster'] != ncid]

    print(f"{len(valid_orders_df)} orders still are valid and can be taken")
    return ncid,unique_clusters,valid_orders_df


def collect_orders(nearest_cluster,nearest_cluster_orders, vehicle_capacity,orders_df,current_point,mode='nearest_cluster',distance_threshold = 5):
    """This function collects orders until the vehicle capacity is reached, either from the nearest cluster or from clusters within a specified distance threshold."""
    collected_orders = []
    exceeds_weight_orders = []
    remaining_capacity = vehicle_capacity

    orders_df = remove_collected_orders(orders_df, collected_orders)

    collected_orders,exceeds_weight_orders,remaining_capacity,current_point = nearest_cluster_collection(nearest_cluster_orders,
                                                                                                         exceeds_weight_orders,
                                                                                                         remaining_capacity,
                                                                                                         collected_orders,current_point)
    # print all uncollected orders at once and restart list
    exceeds_weight_orders = print_uncollected_orders(exceeds_weight_orders)

    # update orders df
    orders_df = remove_collected_orders(orders_df, collected_orders)

    # 2. Mode nearest cluster try to get the order only of nearest cluster
    if mode == 'nearest_cluster':
      # order by distance point again
      try:
        # reorder it as now its a different point - last point visited on first cluster
        orders_df['distance_to_leaving_point'] = orders_df['coordinates'].apply(lambda x: haversine(x, current_point))
        orders_df = orders_df.sort_values(by='distance_to_leaving_point')

        valid_orders_df = orders_df

        unique_clusters = orders_df['cluster'].unique()

        ncid,unique_clusters,valid_orders_df = remove_collected_cluster(nearest_cluster,unique_clusters,valid_orders_df)

        print(unique_clusters, 'unique clusters left.')

        nearest_cluster, nearest_cluster_orders = get_nearest_cluster_orders(valid_orders_df,current_point)

        collected_orders,exceeds_weight_orders,remaining_capacity,current_point = nearest_cluster_collection(nearest_cluster_orders,
                                                                                                             exceeds_weight_orders,
                                                                                                             remaining_capacity,
                                                                                                             collected_orders,current_point)
        exceeds_weight_orders = print_uncollected_orders(exceeds_weight_orders)

      except TypeError as e:
        print(f"no other cluster available: {e}")

      orders_df = remove_collected_orders(orders_df, collected_orders)
      print(f'remaining_capacity: {remaining_capacity}')

    # 3. try to get orders from the following nearest clusters
    if mode == 'distance_threshold':

      # filter orders valid in terms of distance threshold
      # reorder it as now its a different point - last point visited on first cluster
      orders_df['distance_to_leaving_point'] = orders_df['coordinates'].apply(lambda x: haversine(x, current_point))

      valid_orders_df = orders_df[orders_df['distance_to_leaving_point'] <= distance_threshold]
      high_threshold_orders = orders_df[orders_df['distance_to_leaving_point'] > distance_threshold]
      exceeds_weight_orders = print_uncollected_orders(exceeds_weight_orders)
      unique_clusters = valid_orders_df['cluster'].unique()

      ncid,unique_clusters,valid_orders_df = remove_collected_cluster(nearest_cluster,unique_clusters,valid_orders_df)

      print(unique_clusters, 'unique clusters with orders matching distance threshold.')

      while len(unique_clusters) >= 1:
        try:

          nearest_cluster, nearest_cluster_orders = get_nearest_cluster_orders(valid_orders_df,current_point)

          collected_orders,exceeds_weight_orders,remaining_capacity,current_point = nearest_cluster_collection(nearest_cluster_orders,
                                                                                                               exceeds_weight_orders,
                                                                                                               remaining_capacity,
                                                                                                               collected_orders,current_point)
          # print all uncollected orders at once and restart list
          exceeds_weight_orders = print_uncollected_orders(exceeds_weight_orders)

          ncid,unique_clusters,valid_orders_df = remove_collected_cluster(nearest_cluster,unique_clusters,valid_orders_df)

        except TypeError:
            print("No other cluster available meeting distance threshold.")
            orders_df = remove_collected_orders(orders_df, collected_orders)
            break

      orders_df = remove_collected_orders(orders_df, collected_orders)
      print(f'remaining_capacity: {remaining_capacity}')
    return collected_orders,remaining_capacity

def remove_collected_orders(orders_df, collected_orders):
    """Remove collected orders from the data."""
    orders_df = orders_df[~orders_df['order_id'].isin(collected_orders)]
    return orders_df

def remove_vehicle(vehicle_df, vehicle_id):
    """Remove vehicle orders from the data."""
    vehicle_df = vehicle_df[vehicle_df['vehicle_id']!=vehicle_id]
    return vehicle_df

def plot_routes_with_points(df_results, orders_df, depot_coordinates, filename='routes_map_with_points.html'):
    """Plot routes and points for each truck on a folium map and save it to an HTML file."""
     #Create a map centered around the depot
    m = folium.Map(location=depot_coordinates, zoom_start=10)#,tiles='Stamen Terrain'

    # Define colors for each route
    num_trucks = len(df_results)
    colormap = cm.get_cmap('tab20b', num_trucks)
    # Plot points for each order
    for idx, row in orders_df.iterrows():
        folium.CircleMarker(location=(row['latitude'], row['longitude']), radius=5, color='black', fill=True, fill_color='black').add_to(m)

    legend_html = '<div style="position: fixed; top: 50px; left: 50px; z-index:9999; font-size: 14px; background-color:white; border-radius: 5px; padding: 10px;">'
    # Plot each route
    for idx, row in df_results.iterrows():
        vehicle_id = row['vehicle_id']
        orders = row['orders']
        route_coordinates = [depot_coordinates]  # Start with depot
        print(vehicle_id,orders,route_coordinates)
        for order_id in orders:
            order = orders_df.loc[orders_df['order_id'] == order_id]
            # print(order)
            order_coordinates = (order['latitude'].values[0], order['longitude'].values[0])
            route_coordinates.append(order_coordinates)
        route_coordinates.append(depot_coordinates)  # End at depot

        # Plot route on map with color for each truck
        color = colors.rgb2hex(colormap(idx)[:3])
        folium.PolyLine(locations=route_coordinates, color=color, weight=2.5, opacity=1).add_to(m)
        legend_html += f'<div style="margin-bottom: 5px;"><i class="fa fa-square" style="color:{color}"></i> Vehicle {vehicle_id}</div>'
    legend_html += '</div>'

    m.get_root().html.add_child(folium.Element(legend_html))
    # Save map to HTML file
    # m.save(filename)
    return m

def perform_clustering(orders_df, vehicle_df,mode):

    #This function performs clustering on orders and adjusts the number of clusters based on the mode provided.
    if mode == 'distance_threshold':
        try:
            orders_df, kmeans = run_kmeans(orders_df, vehicle_df, num_clusters=len(vehicle_df))
        except ValueError:
            print('All points collected, routing ended')
            return None, None

    elif mode == 'nearest_cluster':
        try:
            print('Increasing number of clusters due to low order density')
            orders_df, kmeans = run_kmeans(orders_df, vehicle_df, num_clusters=len(vehicle_df) +1 )
        except ValueError:
            print('All points collected, routing ended')
            return None, None
    else:
        print(f"Invalid mode '{mode}'. Please choose 'distance_threshold' or 'nearest_cluster'.")
        return None, None
    return orders_df, kmeans

def update_results_df(df_results,vehicle_id,collected_orders,vehicle_capacity,remaining_capacity):
  """This function updates the results dataframe with information about the assigned vehicle, collected orders, and remaining capacity. """
  new_row = {
      'vehicle_id': vehicle_id,
      'orders': collected_orders,
      'vehicle_capacity': vehicle_capacity,
      'remaining_capacity':remaining_capacity
      }

  df_results.loc[len(df_results)] = new_row
  return df_results

def calculate_total_distance(df_results, orders_df,leaving_point):
    """ This function calculates the total distance traveled by each vehicle."""
    total_distances = {}
    for idx, row in df_results.iterrows():
        vehicle_id = row['vehicle_id']
        orders = row['orders']
        total_distance = 0

        current_location = leaving_point  # Start from the depot
        for order_id in orders:
            order_location = orders_df.loc[orders_df['order_id'] == order_id, ['latitude', 'longitude']].values[0]
            distance = haversine(current_location, order_location)
            total_distance += distance
            current_location = order_location

        # Return to the depot
        distance_to_depot = haversine(current_location, leaving_point)
        total_distance += distance_to_depot

        total_distances[vehicle_id] = total_distance

    return total_distances


## Data reading 1

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# orders with 2 special care
data_file_path = 'drive/My Drive/VRP/Orders.csv'

orders_df = pd.read_csv(data_file_path)

# full vehicle
vehicle_file_path = 'drive/My Drive/VRP/vehicle_data.csv'

vehicle_df = pd.read_csv(vehicle_file_path)

Mounted at /content/drive


## Clustering Options

### Mode 1 - Distance threshold

In [ ]:
orders_df= pd.read_csv(data_file_path)
vehicle_df = pd.read_csv(vehicle_file_path)

columns = ['vehicle_id', 'orders', 'vehicle_capacity', 'remaining_capacity']

# Create an empty DataFrame
df_results = pd.DataFrame(columns=columns)
num_orders = len(orders_df)
# while there is a vehicle available, do the collection
while len(vehicle_df)>=1:
  vehicle_id = vehicle_df.iloc[0]['vehicle_id']
  print(f'\n\n collecting for vehicle {vehicle_id}\n')


  orders_df = combine_lat_lon(orders_df)

  orders_df,kmeans = perform_clustering(orders_df, vehicle_df,"distance_threshold")

  if kmeans == None:
    print('all possible points are collected, routing ended')
    break
  leaving_point = (52.499773, -2.024925)

  print_clusters(orders_df, kmeans)
  print("points that need to be visited")
  m = plot_clusters(orders_df,leaving_point)
  display(m)

  nearest_cluster, nearest_cluster_orders = get_nearest_cluster_orders(orders_df,leaving_point)
  vehicle_id, vehicle_capacity = assign_vehicle(nearest_cluster, vehicle_df)
  print(f'vehicle capacity is {vehicle_capacity}')

  collected_orders,remaining_capacity = collect_orders(nearest_cluster,nearest_cluster_orders,
                                                       vehicle_capacity,orders_df,
                                                       leaving_point,
                                                       mode='distance_threshold',
                                                       distance_threshold = 24)

  df_results = update_results_df(df_results,vehicle_id,collected_orders,vehicle_capacity,remaining_capacity)

  orders_df = remove_collected_orders(orders_df, collected_orders)

  vehicle_df = remove_vehicle(vehicle_df, vehicle_id)

  print("Collected Orders:", collected_orders)
  print("Vehicle Assigned:", vehicle_id)


In [ ]:

print (f" from {num_orders} orders, {num_orders - len(orders_df)} - {100*((num_orders - len(orders_df))/num_orders)}% were collected, using all vehicles and {len(orders_df)} orders were not collected \n\n")
orders_df

#### post processing:
if any special care were left out, get the closest collected points from it and if it is not a special care case , keep removing from route, until the point can be collected

In [ ]:
uncollected_points = orders_df.copy()

uncollected_points

In [ ]:
special_care_uncollected = uncollected_points[uncollected_points['special_care']==1]

special_care_uncollected

In [ ]:
orders_df= pd.read_csv(data_file_path)
collected_points = orders_df[~orders_df.isin(uncollected_points)].dropna()

In [ ]:
df_results_preprocessed =  df_results.copy()
df_results_preprocessed

In [ ]:
df_results = df_results_preprocessed.copy()

if len(special_care_uncollected) >= 1:

  collected_points = combine_lat_lon(collected_points)

  # for all points not collected, find nearest points at collected orders
  for idx, order in special_care_uncollected.iterrows():
        display(special_care_uncollected)
        # inicialize var to only collect one point
        index_to_collect = -1
        id_special_care_order = order['order_id']
        order_special_weight = order['order_weight']
        print(f'\n \n trying to add order {id_special_care_order} with weight {order_special_weight}')

        collected_points['distance_to_leaving_point'] = collected_points['coordinates'].apply(lambda x: haversine(x, order['coordinates']))
        collected_points = collected_points.sort_values(by='distance_to_leaving_point')
        collected_points['order_id'] = collected_points['order_id'].astype(int)
        print('closest orders')
        display(collected_points)

        # display(df_results)
        for index_collected, order_collected in collected_points.iterrows():
          if index_to_collect == -1:
            if order_collected['special_care'] == 0:
              index_to_collect = index_collected

              order_id =  order_collected['order_id']
              rm_order_weight = order_collected['order_weight']
              print(f'try to remove order {order_id}')

        for index, row in df_results.iterrows():
          print( row['orders'])
          if order_id in row['orders']:
            print(index,row['orders'])
            collected_by_truck = row['vehicle_id']
            final_truck_weight =  row['remaining_capacity']
            truck_capacity =  row['vehicle_capacity']
            after_collecting_weight = final_truck_weight + rm_order_weight - order_special_weight
            print(f'need to collect {order_special_weight} \n analysing order id {order_id} with weight {rm_order_weight} \n for truck {collected_by_truck} that previously ended up with {final_truck_weight} and can carry {truck_capacity} leaving a weight of {after_collecting_weight}')
            print(after_collecting_weight)
            if after_collecting_weight >= 0:
              print("collecting point")
              print( row['orders'])
              # Find the index of the order_id previously collected
              index_of_previous_order = row['orders'].index(order_id)

              # Replace index_of_previous_order with id_special_care_order
              row['orders'][index_of_previous_order] = id_special_care_order
              print( row['orders'])

              df_results.loc[df_results['vehicle_id'] == collected_by_truck, 'remaining_capacity'] = after_collecting_weight


            else:
              print("can't collect point, leaving order as uncollected")



#### Plot route

In [ ]:
# new new
orders_df= pd.read_csv(data_file_path)

orders_df['order_id'] = orders_df['order_id'].astype(int)

# Call the function to plot the routes
#lot_routes_with_points(df_results, orders_df, depot_coordinates=(52.499773, -2.024925))

In [ ]:
orders_df= pd.read_csv(data_file_path)

orders_df = combine_lat_lon(orders_df)

total_distances = calculate_total_distance(df_results, orders_df,leaving_point)

df_results['total_distance (mi)'] = df_results['vehicle_id'].map(total_distances)

total_distance = sum(df_results['total_distance (mi)'])

print(f'total distance travelled: {total_distance} miles')

total_remaining_capacity = sum(df_results['remaining_capacity'])

print(f'total capacity left: {total_remaining_capacity}')

df_results

In [ ]:
df_results.to_csv('drive/My Drive/VRP/vehicle_allocation_distance_treshold.csv',index=False)

### Mode 2 - Only one nearest cluster

In [ ]:
orders_df= pd.read_csv(data_file_path)
vehicle_df = pd.read_csv(vehicle_file_path)

columns = ['vehicle_id', 'orders', 'vehicle_capacity', 'remaining_capacity']

# Create an empty DataFrame
df_results = pd.DataFrame(columns=columns)
num_orders = len(orders_df)
# while there is a vehicle available, do the collection
while len(vehicle_df)>=1:
  vehicle_id = vehicle_df.iloc[0]['vehicle_id']
  print(f'\n\n collecting for vehicle {vehicle_id}\n')

  orders_df = combine_lat_lon(orders_df)
  orders_df,kmeans = perform_clustering(orders_df, vehicle_df,"nearest_cluster")

  if kmeans == None:
    print('all possible points are collected, routing ended')
    break

  leaving_point = (52.499773, -2.024925)

  print_clusters(orders_df, kmeans)
  print("points that need to be visited")
  m = plot_clusters(orders_df,leaving_point)
  display(m)

  nearest_cluster, nearest_cluster_orders = get_nearest_cluster_orders(orders_df,leaving_point)
  vehicle_id, vehicle_capacity = assign_vehicle(nearest_cluster, vehicle_df)
  print(f'vehicle capacity is {vehicle_capacity}')

  collected_orders,remaining_capacity = collect_orders(nearest_cluster,nearest_cluster_orders,
                                                       vehicle_capacity,orders_df,
                                                       leaving_point,
                                                       mode='nearest_cluster')

  df_results = update_results_df(df_results,vehicle_id,collected_orders,vehicle_capacity,remaining_capacity)

  orders_df = remove_collected_orders(orders_df, collected_orders)

  vehicle_df = remove_vehicle(vehicle_df, vehicle_id)

  print("Collected Orders:", collected_orders)
  print("Vehicle Assigned:", vehicle_id)


In [ ]:

print (f" from {num_orders} orders, {num_orders - len(orders_df)} - {100*((num_orders - len(orders_df))/num_orders)}% were collected, using all vehicles and {len(orders_df)} orders were not collected \n\n")
orders_df

#### post processing:
if any special care were left out, get the closest collected points from it and if it is not a special care case , keep removing from route, until the point can be collected

In [ ]:
uncollected_points = orders_df.copy()

uncollected_points

In [ ]:
special_care_uncollected = uncollected_points[uncollected_points['special_care']==1]

special_care_uncollected

In [ ]:
orders_df= pd.read_csv(data_file_path)
collected_points = orders_df[~orders_df.isin(uncollected_points)].dropna()

In [ ]:
df_results_preprocessed =  df_results.copy()
df_results_preprocessed

In [ ]:
df_results = df_results_preprocessed.copy()

if len(special_care_uncollected) >= 1:

  collected_points = combine_lat_lon(collected_points)

  # for all points not collected, find nearest points at collected orders
  for idx, order in special_care_uncollected.iterrows():
        display(special_care_uncollected)
        # inicialize var to only collect one point
        index_to_collect = -1
        id_special_care_order = order['order_id']
        order_special_weight = order['order_weight']
        print(f'\n \n trying to add order {id_special_care_order} with weight {order_special_weight}')

        collected_points['distance_to_leaving_point'] = collected_points['coordinates'].apply(lambda x: haversine(x, order['coordinates']))
        collected_points = collected_points.sort_values(by='distance_to_leaving_point')
        collected_points['order_id'] = collected_points['order_id'].astype(int)
        print('closest orders')
        display(collected_points)

        # display(df_results)
        for index_collected, order_collected in collected_points.iterrows():
          if index_to_collect == -1:
            if order_collected['special_care'] == 0:
              index_to_collect = index_collected

              order_id =  order_collected['order_id']
              rm_order_weight = order_collected['order_weight']
              print(f'try to remove order {order_id}')

        for index, row in df_results.iterrows():
          print( row['orders'])
          if order_id in row['orders']:
            print(index,row['orders'])
            collected_by_truck = row['vehicle_id']
            final_truck_weight =  row['remaining_capacity']
            truck_capacity =  row['vehicle_capacity']
            after_collecting_weight = final_truck_weight + rm_order_weight - order_special_weight
            print(f'need to collect {order_special_weight} \n analysing order id {order_id} with weight {rm_order_weight} \n for truck {collected_by_truck} that previously ended up with {final_truck_weight} and can carry {truck_capacity} leaving a weight of {after_collecting_weight}')
            print(after_collecting_weight)
            if after_collecting_weight >= 0:
              print("collecting point")
              print( row['orders'])
              # Find the index of the order_id previously collected
              index_of_previous_order = row['orders'].index(order_id)

              # Replace index_of_previous_order with id_special_care_order
              row['orders'][index_of_previous_order] = id_special_care_order
              print( row['orders'])

              df_results.loc[df_results['vehicle_id'] == collected_by_truck, 'remaining_capacity'] = after_collecting_weight


            else:
              print("can't collect point, leaving order as uncollected")



In [ ]:
orders_df= pd.read_csv(data_file_path)

orders_df = combine_lat_lon(orders_df)

total_distances = calculate_total_distance(df_results, orders_df,leaving_point)

df_results['total_distance (mi)'] = df_results['vehicle_id'].map(total_distances)

total_distance = sum(df_results['total_distance (mi)'])

print(f'total distance travelled: {total_distance} miles')

total_remaining_capacity = sum(df_results['remaining_capacity'])

print(f'total capacity left: {total_remaining_capacity}')

df_results

In [ ]:
orders_df['order_id'] = orders_df['order_id'].astype(int)

# Call the function to plot the routes
plot_routes_with_points(df_results, orders_df, depot_coordinates=(52.499773, -2.024925))
df_results.to_csv('drive/My Drive/VRP/vehicle_allocation_nearest_cluster.csv',index=False)

## Data reading 2

In [ ]:
results_file_path = 'drive/My Drive/VRP/vehicle_allocation_nearest_cluster.csv'

results_df =  pd.read_csv(results_file_path)

data_file_path = 'drive/My Drive/VRP/Orders.csv'

orders_df = pd.read_csv(data_file_path)

authenticate api

In [ ]:
API_key = '' #enter your google maps api key here


In [ ]:
import datetime

def seconds_to_hms(seconds):
    timestamp = datetime.datetime.utcfromtimestamp(seconds)
    return timestamp.strftime("%H:%M:%S")

def get_filtered_orders_df(results_df, vehicle_id):
    results_df_filtered = results_df[results_df['vehicle_id'] == vehicle_id]
    print("points to be collected")
    display(results_df_filtered)
    list_orders = ast.literal_eval(results_df_filtered['orders'].iloc[0])
    list_orders.append('depot')
    orders_df_filtered = orders_df.loc[orders_df['order_id'].isin(list_orders)]

    # Add depot
    new_row_data = {'latitude': 52.499773,
                    'longitude': -2.024925,
                    'order_id': 0,
                    'location': 'depot',
                    'order_weight': 0}
    new_row = pd.DataFrame([new_row_data])
    orders_df_filtered = pd.concat([new_row, orders_df_filtered], ignore_index=True)
    return orders_df_filtered, list_orders,new_row,results_df_filtered

def time_constraint():
    distance_list = []
    time_list = []
    row_depot =  orders_df_filtered[orders_df_filtered['order_id']==0]
    dest_depot = (row_depot['latitude'], row_depot['longitude'])
    total_time = 0
    total_distance = 0
    indices_included = 0
    # add time back to base
    for idx,order_ in enumerate(list_orders):

      sum_until_six_hours = 0
      print(idx,order_,len(list_orders))

      if idx == len(list_orders)-1:
        # if its the last order, origin is the point of the list matching order
        # and destination is the depot anyway
        print("last order, go to depot")
        row_orig = orders_df_filtered[orders_df_filtered['order_id']==list_orders[idx-1]]
        row_dest= row_depot
      else:
        if idx == 0:
          # if its the first order, origin is the depot and
          # destination is the point of the list matching order
          print("first order, go from depot")
          row_orig = row_depot
          row_dest= orders_df_filtered[orders_df_filtered['order_id']==list_orders[idx]]

        else:
          # if its the some order in the middle, origin is  the point of the list matching order
          # destination is the next point of the list matching order
          print("intermediate order")
          row_orig = orders_df_filtered[orders_df_filtered['order_id']==list_orders[idx-1]]
          row_dest= orders_df_filtered[orders_df_filtered['order_id']==list_orders[idx]]

      origin = (row_orig['latitude'], row_orig['longitude'])
      dest = (row_dest['latitude'], row_dest['longitude'])
      # display(row_orig)
      # display(row_dest)
      print('\n\n')
      # if a percentege of the time is taken, calculate time to come back
      # if after adding it reaches 90% of the hours, add it.
      # else, keep routing
      if total_time >= percentage_time_filling*hours:
        print("calculating time to go back")

        go_back_query = gmaps.distance_matrix(origin, dest_depot, mode='driving')
        go_back_result_distance = go_back_query["rows"][0]["elements"][0]["distance"]["value"]
        # distance_list.append(result_distance)
        go_back_result_time = go_back_query["rows"][0]["elements"][0]["duration"]["value"]
        # time_list.append(result_time)
        if total_time + go_back_result_time <= max_hours*60*60:
          print(f"{total_time} + {go_back_result_time} dont exceed {max_hours*60*60}, keeping routing")
          query = gmaps.distance_matrix(origin, dest, mode='driving')

          result_distance = query["rows"][0]["elements"][0]["distance"]["value"]
          distance_list.append(result_distance)
          result_time = query["rows"][0]["elements"][0]["duration"]["value"]
          # result_time = result_time/3600
          time_list.append(result_time)
          total_time = total_time + result_time
          total_distance = total_distance + result_distance
          print(f"added {result_time} to total time and {result_distance} to total distance")
          indices_included = indices_included + 1

        else:
          print(f"{total_time + go_back_result_time} exceed {max_hours*60*60}, going to depot")
          distance_list.append(go_back_result_distance)
          time_list.append(go_back_result_time)
          total_time = total_time + go_back_result_time
          total_distance = total_distance + go_back_result_distance
          print(f"added {go_back_result_time} to total time and {go_back_result_distance} to total distance")
          print(f"Sum until 6 hours:{total_time} seconds, {total_time/60/60} hours")
          print(f"total distance: {total_distance}")
          print("Indices of items included:", indices_included)
          return distance_list, time_list, total_time, total_distance, indices_included
          # break
      else:
        print(f"{total_time} hours <= {percentage_time_filling*hours} hours , keeping usual routing")
        query = gmaps.distance_matrix(origin, dest, mode='driving')

        result_distance = query["rows"][0]["elements"][0]["distance"]["value"]
        distance_list.append(result_distance)
        result_time = query["rows"][0]["elements"][0]["duration"]["value"]
        # result_time = result_time/3600
        time_list.append(result_time)
        total_time = total_time + result_time
        total_distance = total_distance + result_distance
        print(f"added {result_time} to total time and {result_distance} to total distance")
        indices_included = indices_included + 1

      print(f"Sum until 6 hours:{total_time} seconds, {total_time/60/60} hours")
      print(f"total distance: {total_distance}")
      print("Indices of items included:", indices_included)
    return distance_list, time_list, total_time, total_distance, indices_included

def google_api_distance_matrix():

  #input: CSV file with id,latitude, longitude and capacities
  # desired output: list with matrix distance for each point consumed by or tools
  # start1 = time.time()
  df = orders_df_filtered_time.copy()
  df['order_weight'] = df['order_weight'].astype(int)
  #print(df)
  #empty list - will be used to store calculated distances
  time_list = []
  distance_list = []
  origin_id_list = []
  destination_id_list = []

  for (i1, row1) in df.iterrows():
    #print("origin")
    #print(row1['ID'])
    LatOrigin = row1['latitude']
    LongOrigin = row1['longitude']
    origin = (LatOrigin, LongOrigin)
    origin_id = row1['order_id']
    for (i2, row2) in  df.iterrows():
      #print("destination id")
      #print(row2['ID'])
      LatDestination = row2['latitude']
      LongDestination = row2['longitude']
      destination_id = row2['order_id']
      destination = (LatDestination, LongDestination)
      # print(destination,destination_id)
      try:
          result = gmaps.distance_matrix(origin, destination, mode='driving')
          #uncomment for cool api logs
          #print(result)
          result_distance = result["rows"][0]["elements"][0]["distance"]["value"]
          result_time = result["rows"][0]["elements"][0]["duration"]["value"]
          time_list.append(result_time)
          distance_list.append(result_distance)
          origin_id_list.append(origin_id)
          destination_id_list.append(destination_id)
          #print(df)
      except Exception as e :
        print(f'Error consulting API: {e}')

  size=(len(df.latitude))
  # Input list initialization
  Input = distance_list

  # list of length in which we have to split
  number_points = len(df['order_id'])
  print("Number of points: ",number_points)

  length_to_split = number_points*[number_points]
  #print(length_to_split)

  # Using islice
  Inputt = iter(Input)
  distance_matrix = [list(islice(Inputt, elem))
            for elem in length_to_split]
  # stop1 = time.time()
  # Printing Output
  print("API list", Input)
  #print("Split length list: ", length_to_split)
  print("List of Lists", distance_matrix)

  # Input list initialization
  Input = time_list

  # list of length in which we have to split
  number_points = len(df['order_id'])
  print("Number of points: ",number_points)

  length_to_split = number_points*[number_points]
  #print(length_to_split)

  # Using islice
  Inputt = iter(Input)
  duration_matrix = [list(islice(Inputt, elem))
            for elem in length_to_split]
  duration_matrix
  return distance_matrix, duration_matrix

def create_data_model():
    """Stores the data for the problem."""
    data = {}

    #always 1 number bigger, because it has to return to base
    data['distance_matrix'] = distance_matrix

    capacities = df['order_weight']
    print(capacities)

    data['demands'] = df['order_weight'].values.tolist()
    print(data['demands'])

    data['vehicle_capacities'] = results_df_filtered['vehicle_capacity']
    data['num_vehicles'] = 1

    data['depot'] = 0
    print("Input OR-Tools: ",data," ",type(data))
    return data

def extract_solution(data, manager, routing, solution):
    """Extracts solution data."""
    routes_data = []
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route_data = []
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            route_data.append(node_index)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            print(route_distance)
            route_distance_add = routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
            print(f'current distance: {route_distance}, adding {route_distance_add} for node {node_index}')
            route_distance += route_distance_add
        route_data.append(manager.IndexToNode(index))
        route_data.append(route_load)
        print(route_distance)
        route_data.append(route_distance)
        routes_data.append(route_data)
        total_distance += route_distance
        total_load += route_load
    total_data = {'Route': [], 'Distance': [], 'Load': [], 'Points': []}
    for idx, route in enumerate(routes_data):
        total_data['Route'].append(f'Route {idx}')
        total_data['Distance'].append(route[-1])
        total_data['Load'].append(route[-2])
        total_data['Points'].append(route[:-2])  # Remove load and distance
    return pd.DataFrame(total_data)

def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')
    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(1)
    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    # Print solution on console.
    if solution:
        solution_df = extract_solution(data, manager, routing, solution)
        display(solution_df)
    else:
        print("no solution found")
    return data, manager, routing, solution_df

# Function to compute total duration for a given route
def compute_total_duration(route, duration_matrix):
    total_duration = 0
    for i in range(len(route) - 1):
        from_order = route[i]
        to_order = route[i + 1]
        # print(from_order,to_order,duration_matrix[from_order][to_order])
        total_duration += duration_matrix[from_order][to_order]
        # print(total_duration)
    return total_duration

def update_results_df(results_df_filtered, list_orders_filtered, total_distance, total_time, reordered_list_orders, solution_df, total_duration):
    """
    Updates the results DataFrame with clustering and ORTools results.
    """
    total_distance_miles = "{:.2f}".format(total_distance * 0.000621371)
    distance_ortools = "{:.2f}".format(solution_df['Distance'].iloc[0] * 0.000621371)
    results_df_filtered['remaining_capacity'] = int(results_df_filtered['vehicle_capacity']) - int(solution_df['Load'])
    results_df_filtered['orders_clustering'] = str(list_orders_filtered)
    results_df_filtered['distance_clustering'] = f"{total_distance_miles} mi"
    results_df_filtered['time_clustering'] = str(seconds_to_hms(total_time))
    results_df_filtered['orders_ortools'] = str(reordered_list_orders)
    results_df_filtered['distance_ortools'] = f"{distance_ortools} mi"
    results_df_filtered['time_ortools'] = str (seconds_to_hms(total_duration))

    try:
        results_df_filtered.drop(columns=['orders', 'total_distance (mi)'], inplace=True)
    except KeyError:
        print("already dropped columns")

    return results_df_filtered

def plot_routes_clustering(orders_df_filtered_time, list_orders_filtered):
    """
    Plot routes on Google Maps based on the provided orders DataFrame and list of filtered orders.

    Args:
    - orders_df_filtered_time (pd.DataFrame): DataFrame containing order data with timestamps.
    - list_orders_filtered (list): List of filtered order IDs.

    Returns:
    - fig (gmaps.figure): Google Maps figure with plotted routes.
    """
    # Fetch all necessary data upfront
    points_data = orders_df_filtered_time[orders_df_filtered_time['order_id'].isin(list_orders_filtered)]

    # Rearrange the filtered data based on the order in list_orders_filtered
    points_data = points_data.set_index('order_id').loc[list_orders_filtered].reset_index()

    points = points_data[['latitude', 'longitude']].values.tolist()
    # Start with the depot
    points.insert(0, (52.49, -2.02))  # Depot coordinates
    points_set = [tuple(point) for point in points]

    # Initialize the figure
    fig = gmaps.figure(layout={
        'width': '1200px',
        'height': '1200px',
        'padding': '3px',
        'border': '1px solid black'
    })

    cmap = plt.cm.get_cmap('Dark2', len(points_set))
    colors = [matplotlib.colors.to_rgb(cmap(i)) for i in range(len(points_set))]

    # Plot routes in order
    for i in range(len(points) - 1):
        start_point = points[i]
        end_point = points[i + 1]
        rgb_tuple = tuple(int(x * 255) for x in colors[i])
        route = gmaps.directions_layer(start_point, end_point, travel_mode='DRIVING', show_route=True,
                                        stroke_color=rgb_tuple, show_markers=False)
        try:
          fig.add_layer(route)
        except AssertionError:
          print("too many points to plot in the map, dividing it")

    # Add the last route back to the depot
    route = gmaps.directions_layer(points[-1], points[0], travel_mode='DRIVING', show_route=True, show_markers=False)
    try:
      fig.add_layer(route)
    except AssertionError:
      print("too many points to plot in the map, dividing it")

    labels = [str(point) for point in list_orders_filtered]
    if "Depot" not in labels:
        labels.insert(0, "Depot")  # Depot coordinates
    marker_layer = gmaps.marker_layer(
        points_set,
        label=labels,
    )
    fig.add_layer(marker_layer)
    return fig

def plot_routes_ortools(orders_df_filtered_time, reordered_list_orders):
    """
    Plot routes on Google Maps based on the provided orders DataFrame and list of reordered orders.
    Args:
    - orders_df_filtered_time (pd.DataFrame): DataFrame containing order data with timestamps.
    - reordered_list_orders (list): List of reordered order IDs.

    Returns:
    - fig (gmaps.figure): Google Maps figure with plotted routes.
    """
    # Fetch all necessary data upfront
    points_data = orders_df_filtered_time[orders_df_filtered_time['order_id'].isin(reordered_list_orders)]

    # Rearrange the filtered data based on the order in reordered_list_orders
    points_data = points_data.set_index('order_id').loc[reordered_list_orders].reset_index()

    points = points_data[['latitude', 'longitude']].values.tolist()
    # Start with the depot
    points.insert(0, (52.49, -2.02))  # Depot coordinates
    points_set = [tuple(point) for point in points]

    # Initialize the figure
    fig = gmaps.figure(layout={
        'width': '1200px',
        'height': '1200px',
        'padding': '3px',
        'border': '1px solid black'
    })

    cmap = plt.cm.get_cmap('Dark2', len(points_set))
    colors = [matplotlib.colors.to_rgb(cmap(i)) for i in range(len(points_set))]

    # Plot routes in order
    for i in range(len(points) - 1):
        start_point = points[i]
        end_point = points[i + 1]
        rgb_tuple = tuple(int(x * 255) for x in colors[i])
        route = gmaps.directions_layer(start_point, end_point, travel_mode='DRIVING', show_route=True,
                                        stroke_color=rgb_tuple, show_markers=False)
        try:
          fig.add_layer(route)
        except AssertionError:
          print("too many points to plot in the map, dividing it")
    # Add the last route back to the depot
    route = gmaps.directions_layer(points[-1], points[0], travel_mode='DRIVING', show_route=True, show_markers=False)
    try:
      fig.add_layer(route)
    except AssertionError:
      print("too many points to plot in the map, dividing it")

    labels = [str(point) for point in reordered_list_orders]
    if "Depot" not in labels:
        labels.insert(0, "Depot")  # Depot coordinates
    marker_layer = gmaps.marker_layer(
        points_set,
        label=labels,
    )
    fig.add_layer(marker_layer)

    return fig

### choose vehicles ids to perform routing

In [ ]:
vehicle_ids_list = [100,101,102,103,104]

In [ ]:
# select True to run just time constraint part with few requests and not full distance matrix - good for testing
run_just_time = False

# select True to plot and False to skip this part
plot_routes = False
# Iterate over vehicle ids
for vehicle_id in vehicle_ids_list:
    print("\n\n\n\n--------------------------------------------------------------")
    print(f"Performing route comparison for vehicle {vehicle_id}")
    import googlemaps
    try:
        gmaps = googlemaps.Client(key=API_key)
    except ValueError as e :
        print(f'Error consulting API: {e}')
    # Get filtered orders dataframe and list of orders
    orders_df_filtered, list_orders,new_row,results_df_filtered = get_filtered_orders_df(results_df, vehicle_id)

    print("Points to be collected:")
    #display(orders_df_filtered)
    # display(list_orders)

    # Compute route duration
    hours = 8
    # minimum percentage to check how long it takes to go back to depot
    percentage_time_filling = 0.7
    # time to stop routing 0.1 is stopping when it reaches 0.6 hour
    max_hours = hours

    distance_list, time_list, total_time, total_distance, indices_included = time_constraint()

    list_orders_filtered = list_orders[:indices_included]
    if 'depot' in list_orders_filtered:
      list_orders_filtered.remove('depot')

    print(f'\n\n list_orders filtered with with time constraint is{list_orders_filtered} from {list_orders}')

    print("rebuilding dataframe of orders, now it is:\n")
    orders_df_filtered_time = orders_df_filtered.loc[orders_df_filtered['order_id'].isin(list_orders_filtered)]
    orders_df_filtered_time = pd.concat([new_row, orders_df_filtered_time], ignore_index=True)
    #display(orders_df_filtered_time)

    print("\n\n calculating distance matrix for remaining points")
    if run_just_time == True:
      print("stopping and running just time")
    else:
      distance_matrix, duration_matrix =  google_api_distance_matrix()

      print(f'distance matrix: \n {distance_matrix} \n time matrix: \n {duration_matrix}')

      print("\n\n calculating ortools with distance matrix")
      Output = distance_matrix
      num_points = len(distance_matrix)
      print(num_points)

      df =  orders_df_filtered_time.copy()

      df['order_weight'] = df['order_weight'].astype(int)

      data, manager, routing, solution_df = main()
      collected_points = solution_df['Points']
      for i in collected_points:
        result_ortools = i

      result_ortools.pop(0)

      result_ortools.pop(-1)

      # reordered_list_orders = [list_orders_filtered[index - 1] for index in result_ortools]
      reordered_list_orders = [orders_df_filtered_time.iloc[idx]['order_id'] for idx in result_ortools]

      reordered_list_orders_calculation = [0] + result_ortools + [0]
      # Compute total duration for the specified route
      total_duration = compute_total_duration(reordered_list_orders_calculation, duration_matrix)
      print("Total duration for the specified route:", total_duration)

      updated_results_df = update_results_df(results_df_filtered, list_orders_filtered, total_distance, total_time, reordered_list_orders, solution_df, total_duration)
      display(updated_results_df)
      if plot_routes == True:
        import gmaps
        try:
            gmaps.configure(api_key=API_key)
        except ValueError as e :
            print(f'Error consulting API: {e}')

        print("clustering route")

        fig = plot_routes_clustering(orders_df_filtered_time, list_orders_filtered)
        display(fig)

        print("ortools route")
        fig =  plot_routes_ortools(orders_df_filtered_time, reordered_list_orders)
        display(fig)
      # break